# Schottky Barrier Diode

## Metal-Semiconductor Junction Physics

Schottky diodes are formed at metal-semiconductor interfaces and offer advantages over PN junctions including faster switching and lower forward voltage drop.

**Learning Objectives:**
- Understand Schottky barrier formation
- Compare with PN junction diodes
- Analyze barrier height effects
- Simulate I-V characteristics

---

## 1. Schottky Barrier Physics

### 1.1 Band Diagram

When a metal contacts an n-type semiconductor:

```
    Metal     |     N-type Semiconductor
              |
    ----      |     ----
    Ef_m      |          \____ Ec
              |                \
              |                 \---- Ef_s
              |                       
              |              \_______ Ev
              |
         <-- Depletion -->
```

### 1.2 Key Parameters

- **Schottky barrier height** ($\phi_B$): Energy barrier at the interface
  $$\phi_B = \phi_m - \chi$$
  where $\phi_m$ is metal workfunction and $\chi$ is semiconductor electron affinity

- **Built-in potential**:
  $$V_{bi} = \phi_B - \frac{kT}{q}\ln\left(\frac{N_c}{N_d}\right)$$

- **Thermionic emission current**:
  $$J = A^* T^2 \exp\left(-\frac{q\phi_B}{kT}\right)\left[\exp\left(\frac{qV}{kT}\right) - 1\right]$$
  where $A^* \approx 120$ A/cm$^2$/K$^2$ is Richardson's constant

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_schottky_diode

# Physical constants
q = 1.6e-19       # Elementary charge (C)
kT = 0.0259       # Thermal voltage at 300K (eV)
chi_si = 4.05     # Silicon electron affinity (eV)
Nc = 2.8e19       # Effective density of states in CB (cm^-3)
A_star = 120      # Richardson constant (A/cm^2/K^2)
T = 300           # Temperature (K)

# Metal workfunctions (eV)
metals = {
    'Ti': 4.33,
    'Cr': 4.50,
    'W': 4.55,
    'Mo': 4.60,
    'Au': 5.10,
    'Pt': 5.65
}

print("Schottky Barrier Heights on n-Si:")
print("="*40)
for metal, phi_m in metals.items():
    phi_B = phi_m - chi_si
    print(f"{metal}: phi_m = {phi_m:.2f} eV, phi_B = {phi_B:.2f} eV")

---

## 2. Creating a Schottky Diode

Let's create Schottky diodes with different barrier heights.

In [ ]:
# Create a basic Schottky diode (Ti contact on n-Si)
sim_ti = create_schottky_diode(
    # Geometry
    length=2.0,          # Device length (um)
    width=1.0,           # Device width (um)
    
    # Doping
    doping=1e17,         # N-type doping (cm^-3)
    doping_type='n',     # N-type substrate
    
    # Contact
    workfunction=4.33,   # Ti workfunction (eV)
    
    # Output
    log_bands_eq=True,
    log_iv=True,
    iv_file="ti_schottky",
    
    # Voltage sweep
    forward_sweep=(0.0, 0.5, 0.02)
)

print("Schottky Diode Configuration")
print("="*40)
print(f"Metal: Ti (phi_m = 4.33 eV)")
print(f"Barrier height: {4.33 - chi_si:.2f} eV")
print(f"Substrate: n-Si (Nd = 1e17 cm^-3)")

In [ ]:
# View the generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_ti.generate_deck())

---

## 3. Barrier Height Effect

The Schottky barrier height strongly affects the I-V characteristics.

In [ ]:
# Create diodes with different barrier heights
barrier_heights = [0.3, 0.5, 0.7, 0.9]  # eV
workfunctions = [chi_si + phi_B for phi_B in barrier_heights]

print("Comparing Different Barrier Heights:")
print("="*50)

for phi_B, phi_m in zip(barrier_heights, workfunctions):
    sim = create_schottky_diode(
        doping=1e17,
        workfunction=phi_m,
        log_iv=True,
        forward_sweep=(0.0, 0.5, 0.02)
    )
    
    # Calculate saturation current density
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    
    print(f"phi_B = {phi_B:.1f} eV: Js = {Js:.2e} A/cm^2")

In [ ]:
# Theoretical I-V curves for different barrier heights
V = np.linspace(0, 0.5, 100)
A = 1e-8  # Device area (cm^2)

plt.figure(figsize=(12, 5))

# Linear scale
plt.subplot(1, 2, 1)
for phi_B in barrier_heights:
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    I = A * Js * (np.exp(V / kT) - 1)
    plt.plot(V, I * 1e6, linewidth=2, label=f'$\phi_B$ = {phi_B} eV')

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (uA)', fontsize=12)
plt.title('Schottky I-V: Effect of Barrier Height', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

# Log scale
plt.subplot(1, 2, 2)
for phi_B in barrier_heights:
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    I = A * Js * (np.exp(V / kT) - 1)
    plt.semilogy(V, np.abs(I), linewidth=2, label=f'$\phi_B$ = {phi_B} eV')

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Schottky I-V (Log Scale)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. Schottky vs PN Junction

Let's compare Schottky and PN junction diodes.

In [ ]:
from nanohubpadre import create_pn_diode

# Schottky diode
sim_schottky = create_schottky_diode(
    doping=1e17,
    workfunction=4.55,  # phi_B ~ 0.5 eV
    log_iv=True,
    forward_sweep=(0.0, 0.8, 0.02)
)

# PN junction diode
sim_pn = create_pn_diode(
    p_doping=1e17,
    n_doping=1e17,
    log_iv=True,
    forward_sweep=(0.0, 0.8, 0.02)
)

print("Comparison: Schottky vs PN Junction")
print("="*50)
print("\nSchottky Diode Advantages:")
print("  - Lower turn-on voltage (~0.3V vs ~0.7V)")
print("  - Faster switching (majority carrier device)")
print("  - No minority carrier storage")
print("\nSchottky Diode Disadvantages:")
print("  - Higher reverse leakage current")
print("  - Lower breakdown voltage")

In [ ]:
# Theoretical comparison
V = np.linspace(0, 0.8, 200)
A = 1e-8  # cm^2

# Schottky (phi_B = 0.5 eV)
phi_B = 0.5
Js_schottky = A_star * T**2 * np.exp(-phi_B / kT)
I_schottky = A * Js_schottky * (np.exp(V / kT) - 1)

# PN junction (typical Is for 1e17 doping)
Is_pn = 1e-14  # A - typical for Si PN diode
I_pn = Is_pn * (np.exp(V / kT) - 1)

plt.figure(figsize=(12, 5))

# Linear
plt.subplot(1, 2, 1)
plt.plot(V, I_schottky * 1e6, 'b-', linewidth=2, label='Schottky')
plt.plot(V, I_pn * 1e6, 'r-', linewidth=2, label='PN Junction')
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (uA)', fontsize=12)
plt.title('Schottky vs PN Junction (Linear)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.8)

# Log
plt.subplot(1, 2, 2)
plt.semilogy(V, np.abs(I_schottky), 'b-', linewidth=2, label='Schottky')
plt.semilogy(V, np.abs(I_pn), 'r-', linewidth=2, label='PN Junction')
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Schottky vs PN Junction (Log)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.8)

# Add turn-on voltage markers
plt.axvline(x=0.3, color='b', linestyle='--', alpha=0.5)
plt.axvline(x=0.7, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print(f"\nTurn-on voltages (at 1uA):")
# Find voltages where current = 1uA
V_on_schottky = kT * np.log(1e-6 / (A * Js_schottky) + 1)
V_on_pn = kT * np.log(1e-6 / Is_pn + 1)
print(f"  Schottky: ~{V_on_schottky:.2f} V")
print(f"  PN Junction: ~{V_on_pn:.2f} V")

---

## 5. Band Diagram Analysis

Examining the band structure under different bias conditions.

In [ ]:
# Simulation with band diagram logging
sim_bands = create_schottky_diode(
    length=1.0,
    doping=1e17,
    workfunction=4.55,
    
    # Log bands at equilibrium and under bias
    log_bands_eq=True,
    log_bands_bias=True,
    
    # Forward and reverse sweeps
    forward_sweep=(0.0, 0.3, 0.1),
    reverse_sweep=(0.0, -2.0, -0.5)
)

print("Band diagram simulation configured")
print("Output files:")
print("  - cbeq, vbeq: Equilibrium bands")
print("  - cbfwd, vbfwd: Forward bias bands")
print("  - cbrev, vbrev: Reverse bias bands")

In [ ]:
# Theoretical band diagrams
def schottky_bands(x, phi_B, Vbi, V_applied, W):
    """Calculate Schottky diode band diagram"""
    Eg = 1.12  # Si bandgap
    Ec = np.zeros_like(x)
    Ev = np.zeros_like(x)
    
    V_eff = Vbi - V_applied
    W_eff = W * np.sqrt(max(V_eff/Vbi, 0.01))
    
    for i, xi in enumerate(x):
        if xi < W_eff:
            # Depletion region (parabolic)
            V_drop = V_eff * (1 - xi/W_eff)**2
            Ec[i] = phi_B - V_applied + V_drop - Eg/2
        else:
            # Neutral region
            Ec[i] = phi_B - Vbi - Eg/2
        Ev[i] = Ec[i] - Eg
    
    return Ec, Ev

# Parameters
phi_B = 0.5
Nd = 1e17
eps_si = 11.7 * 8.85e-14
Vbi = phi_B - kT * np.log(Nc / Nd)
W0 = np.sqrt(2 * eps_si * Vbi / (q * Nd)) * 1e4  # um

x = np.linspace(0, 0.5, 200)
biases = [0, 0.2, -1.0, -2.0]

plt.figure(figsize=(12, 8))
for i, V in enumerate(biases):
    plt.subplot(2, 2, i+1)
    Ec, Ev = schottky_bands(x, phi_B, Vbi, V, W0)
    
    plt.plot(x, Ec, 'b-', linewidth=2, label='Ec')
    plt.plot(x, Ev, 'r-', linewidth=2, label='Ev')
    plt.axhline(y=0, color='g', linestyle='--', label='Ef (metal)')
    
    # Show barrier
    plt.annotate('', xy=(0, phi_B - V), xytext=(0, 0),
                arrowprops=dict(arrowstyle='<->', color='purple'))
    
    plt.xlabel('Position (um)')
    plt.ylabel('Energy (eV)')
    plt.title(f'V = {V} V')
    plt.legend(loc='upper right', fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 0.5)

plt.suptitle('Schottky Diode Band Diagrams', fontsize=14)
plt.tight_layout()
plt.show()

---

## 6. Barrier Lowering Effect

At high electric fields, the Schottky barrier is lowered (image force lowering).

In [ ]:
# Simulation with barrier lowering enabled
sim_barrier = create_schottky_diode(
    doping=1e17,
    workfunction=4.55,
    barrier_lowering=True,  # Enable image force lowering
    log_iv=True,
    forward_sweep=(0.0, 0.4, 0.02),
    reverse_sweep=(0.0, -5.0, -0.25)
)

print("Barrier lowering effect:")
print("  delta_phi = sqrt(qE / 4*pi*eps)")
print("  This reduces the effective barrier at high fields")

---

## 7. Complete Simulation Example

In [ ]:
# Complete Schottky diode characterization
sim_complete = create_schottky_diode(
    # Geometry
    length=2.0,
    width=1.0,
    
    # Mesh
    nx=200,
    ny=3,
    
    # Doping
    doping=1e17,
    doping_type='n',
    
    # Schottky contact
    workfunction=4.55,
    barrier_lowering=True,
    surf_rec=True,
    
    # Models
    temperature=300,
    srh=True,
    conmob=True,
    fldmob=True,
    
    # Output
    log_iv=True,
    iv_file="schottky_complete",
    log_bands_eq=True,
    log_bands_bias=True,
    
    # Sweeps
    forward_sweep=(0.0, 0.5, 0.01),
    reverse_sweep=(0.0, -5.0, -0.1)
)

print("Complete Schottky Diode Simulation")
print("="*50)
print(sim_complete.generate_deck())

---

## 8. Exercises

### Exercise 1: Metal Selection
Create Schottky diodes with Ti, Au, and Pt contacts. Compare their I-V characteristics.

### Exercise 2: Doping Effect
How does substrate doping affect the depletion width and capacitance?

### Exercise 3: Temperature Dependence
The saturation current varies as $T^2 \exp(-q\phi_B/kT)$. Simulate at different temperatures.

In [ ]:
# Exercise 1: Different metals
metal_data = [
    ('Ti', 4.33),
    ('Au', 5.10),
    ('Pt', 5.65)
]

print("Metal Comparison:")
print("="*50)

for metal, phi_m in metal_data:
    phi_B = phi_m - chi_si
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    
    sim = create_schottky_diode(
        doping=1e17,
        workfunction=phi_m,
        log_iv=True,
        iv_file=f"{metal.lower()}_iv",
        forward_sweep=(0.0, 0.6, 0.02)
    )
    
    print(f"\n{metal}:")
    print(f"  Workfunction: {phi_m:.2f} eV")
    print(f"  Barrier height: {phi_B:.2f} eV")
    print(f"  Saturation current density: {Js:.2e} A/cm^2")

---

## Summary

In this notebook, you learned:

1. **Schottky Barrier Formation**: Metal workfunction determines barrier height
2. **Thermionic Emission**: Current mechanism in Schottky diodes
3. **Advantages over PN**: Lower turn-on, faster switching
4. **Band Diagrams**: How bias affects the band structure
5. **Barrier Lowering**: Image force effect at high fields

**Key Equations:**
- Barrier height: $\phi_B = \phi_m - \chi$
- Thermionic current: $J = A^*T^2 e^{-q\phi_B/kT}(e^{qV/kT} - 1)$

**Next**: [04 - MOSFET](04_MOSFET.ipynb) - Field-effect transistors